In [2]:
# import itertools as it
# from multiprocessing import Pool, cpu_count
# from collections import defaultdict
import re
import numpy as np
from functools import reduce
from operator import mul
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^
"""


### Part 1

This seems like bruteforce will do?

It's so cold this morning.

In [76]:
class Sokoban():
    def __init__(self,txt):
        res = txt.split("\n\n")
        self.grid,self.inputs = res[0].splitlines(),res[1]
        self.inputs = ''.join(self.inputs.splitlines())
        self.dirs =  {'>':1j,'v':1,'^':-1,'<':-1j}
        # self.dirs = {'>':(0,1),'v':(1,0),'^':(-1,0),'<':(0,-1)} # ehh the other way is cooler

        self.obs= set()
        self.boxes = set()
        self.bot= 0

        for x in range(len(self.grid)):
            for y in range(len(self.grid[x])):
                # print(i,'',j)
                c=self.grid[x][y]
                if c == '#':
                    self.obs.add(x+y*1j)
                elif c == 'O':
                    self.boxes.add(x+y*1j)
                elif c == '@':
                    self.bot = x+y*1j
        # print(self.obs)
        # print(self.boxes)
        # print(self.bot)
        for input in self.inputs:
            dir = self.dirs.get(input)
            self.move(self.bot,dir)

    def calc_gps_sum(self):
        return int(sum([100*box.real+box.imag for box in self.boxes]))
        
    def char_at_pos(self,pos):
        return self.grid[int(pos.real)][int(pos.imag)]

    def move(self,pos,dir):
        next = pos+dir
        if next in self.obs:
            return False
        if next in self.boxes:
            next_moved = self.move(next,dir)
            if not next_moved:
                return False
        
        if pos in self.boxes:
            self.boxes.remove(pos)
            self.boxes.add(next)
        else:
            self.bot = next
        return True #moved


res = Sokoban(input)
res.calc_gps_sum()


1436690

Trying to understand the complex number coordinate thing

In [50]:
x = 2-3j
print(int(x.real))
print(x.imag)

2
-3.0


### Part 2
Ok man

In [7]:
def x_in_pair_set(x,s): # x for pos, s for boxes
        return next((pair for pair in s if x in pair), None) 

class Sokoban():
    def __init__(self,txt):
        res = txt.split("\n\n")
        self.grid,self.inputs = res[0].splitlines(),res[1]
        self.inputs = ''.join(self.inputs.splitlines())
        self.dirs =  {'>':1j,'v':1,'^':-1,'<':-1j}

        self.obs= set()
        self.boxes = set() # accounts for just the left
        self.bot= 0

        # print(self.grid)
        # print(len(self.grid))
        # print(len(self.grid[0]))

        for x in range(len(self.grid)):
            for y in range(0,len(self.grid[x])*2,2):
                # print(x,'',y)
                c=self.grid[x][y//2]
                if c == '#':
                    self.obs.add(x+(y*1j))
                    self.obs.add(x+(y+1)*1j)
                elif c == 'O':
                    # self.obs.add(x+(y*1j))
                    self.boxes.add(((x+(y*1j)),(x+(y+1)*1j)))
                elif c == '@':
                    self.bot = x+y*1j
        print("obs",(self.obs))
        print("box",(self.boxes))
        print((self.bot))


        for input in self.inputs:
            dir = self.dirs.get(input)
            self.move(self.bot,dir)
            # print(self.obs)
            # print("box",self.boxes)
            # print(self.bot)

    def calc_gps_sum(self):
        return int(sum([100*box[0].real+box[0].imag for box in self.boxes]))
        
    def char_at_pos(self,pos):
        return self.grid[int(pos.real)][int(pos.imag)]

    def move(self,pos,dir):
        next = pos+dir
        if next in self.obs:
            return False
        next_box = x_in_pair_set(next,self.boxes)
        if next_box: # next in self.boxes:
            if dir.real == 0: #L or R
                next_moved = self.move(next+dir,dir)
            else: # U or D
                boxes_copy = set(self.boxes)
                next_moved = self.move_box(next_box,dir,boxes_copy)
                if next_moved: 
                    self.boxes = boxes_copy
            if not next_moved:
                return False
        
        big_box = x_in_pair_set(pos,self.boxes)
        if big_box: #pos in self.boxes:
            if dir.real == 0: #L or R
                self.boxes.remove(big_box)
                self.boxes.add((big_box[0]+dir,big_box[1]+dir))
        else:
            self.bot = next
        return True #moved
    

    def move_box(self,pos,dir,box_locations): # note pos here is a tuple
        next = (pos[0]+dir,pos[1]+dir) # could be 2 boxes!
        # print("          pos=",pos,"  next=",next)

        # check for any next position being obstacles
        if any(boxpos in self.obs for boxpos in next):
            return False
        
        # find possible boxes next positions aligns to
        next_boxes = set([x_in_pair_set(nx,box_locations) for nx in next]) # could be 1 or 2
        
        # recursively check all next boxes until obstacle or free
        for next_box in next_boxes:
            if next_box in box_locations:
                next_moved = self.move_box(next_box,dir,box_locations)
                if not next_moved:
                    # print("          can't move pos=",pos)
                    return False

        # if next boxes are confirmed movable
        if pos in box_locations:
            box_locations.remove(pos)
            box_locations.add(next)
        return True



res = Sokoban(input)
res.calc_gps_sum()


obs {0j, (1+0j), (2+0j), (3+0j), (4+0j), (5+0j), (6+0j), (7+0j), (8+0j), (9+0j), 46j, (10+0j), (11+0j), (12+0j), (13+0j), (14+0j), (6+46j), (7+46j), (15+0j), (16+0j), 92j, (17+0j), (18+0j), (19+0j), (20+0j), (21+0j), (22+0j), (23+0j), (24+0j), (9+92j), (25+0j), (26+0j), (27+0j), (28+0j), (29+0j), (30+0j), (31+0j), (32+0j), (18+92j), (33+0j), (34+0j), (35+0j), (36+0j), (37+0j), (38+0j), (39+0j), (40+0j), (41+0j), (42+0j), (43+0j), (44+0j), (41+46j), (45+0j), 39j, (34+92j), (46+0j), (46+46j), (48+0j), (49+0j), (49+46j), 85j, (27+52j), (45+92j), (49+92j), (7+85j), (26+39j), (24+85j), (37+39j), (31+85j), 32j, (2+32j), (36+85j), (34+52j), (49+39j), 78j, (1+78j), (36+52j), (49+85j), (37+52j), (39+52j), 25j, (2+25j), (47+32j), (49+32j), 71j, (14+25j), (6+71j), (49+78j), (49+52j), 18j, (49+25j), 64j, (44+71j), (46+71j), (48+71j), (49+71j), (12+64j), (22+18j), (25+18j), (20+64j), (35+18j), 11j, (34+64j), (38+64j), (39+64j), (40+64j), (49+18j), (42+64j), 57j, (47+0j), (4+57j), (6+57j), (7+57j), 

1482350

### Part 2
Oh good lord that was miserable

I'm still not sure if rewriting everything would've helped my case? Rewriting the grid for part 2 was fairly straightforward, but then changing the logic ate way too much of my time.

The most difficult part I had to manuver around was the use of recursion. Originally, if a movable block was recursively found it will be moved immediately. This was troublesome because nothing should be movable if any obstacle was found. I ended up passing a set copy in and only replaces the "main" boxes location set if all recursion came back positive. This probably isn't the most effective way, but at this point I'm just glad it worked.

### Takeaways
- Printing -- remember day 14? Yeah. Not printing debug messages cut a crash-worthy time duration (20+seconds) down to 2.
- Take a break and come back is very much worth it when you want to rage quit.
- Part 1 was basically the Sokoban assignment I did in my bachelors! If you are a LEI student at FCUL you might remember this one from IP(Introdução da Programação). So it was a nice refresher + accomplishing to see how far I've come. Part 2 took way longer than I anticipated and it got messy fast -- but I'm still pretty happy to have figured this out.